# RedisVectorStore

```{=mdx}
:::tip 兼容性
仅适用于 Node.js。
:::
```

[Redis](https://redis.io/) 是一个快速的开源内存数据存储。作为 [Redis Stack](https://redis.io/docs/latest/operate/oss_and_stack/install/install-stack/) 的一部分，[RediSearch](https://redis.io/docs/latest/develop/interact/search-and-query/) 是一个支持向量相似性语义搜索以及其他多种搜索方式的模块。

本指南提供了 Redis [向量存储](/docs/concepts/#vectorstores) 的快速入门概览。如需查看所有 `RedisVectorStore` 功能和配置的详细文档，请访问 [API 参考](https://api.js.langchain.com/classes/langchain_redis.RedisVectorStore.html)。

## 概述

### 集成详情

| 类 | 包 | [Python 支持](https://python.langchain.com/docs/integrations/vectorstores/redis/) | 包最新版本 |
| :--- | :--- | :---: | :---: |
| [`RedisVectorStore`](https://api.js.langchain.com/classes/langchain_redis.RedisVectorStore.html) | [`@langchain/redis`](https://npmjs.com/@langchain/redis/) | ✅ |  ![NPM - 版本](https://img.shields.io/npm/v/@langchain/redis?style=flat-square&label=%20&) |

## 环境设置

要使用 Redis 向量存储，你需要设置一个 Redis 实例并安装 `@langchain/redis` 集成包。你还可以安装 [`node-redis`](https://github.com/redis/node-redis) 包，以便使用特定的客户端实例初始化向量存储。

本指南还将使用 [OpenAI 嵌入模型](/docs/integrations/text_embedding/openai)，这需要你安装 `@langchain/openai` 集成包。如果你愿意，也可以使用 [其他支持的嵌入模型](/docs/integrations/text_embedding)。

```{=mdx}
import IntegrationInstallTooltip from "@mdx_components/integration_install_tooltip.mdx";
import Npm2Yarn from "@theme/Npm2Yarn";

<IntegrationInstallTooltip></IntegrationInstallTooltip>

<Npm2Yarn>
  @langchain/redis @langchain/core redis @langchain/openai
</Npm2Yarn>
```

你可以通过 Docker 按照 [这些说明](https://redis.io/docs/latest/operate/oss_and_stack/install/install-stack/docker/#redisredis-stack) 在本地设置一个 Redis 实例。

### 凭证

设置好实例后，设置 `REDIS_URL` 环境变量：

```typescript
process.env.REDIS_URL = "your-redis-url"
```

如果本指南中你使用的是 OpenAI 嵌入模型，还需要设置你的 OpenAI 密钥：

```typescript
process.env.OPENAI_API_KEY = "YOUR_API_KEY";
```

如果你想对模型调用进行自动追踪，还可以通过取消以下注释来设置你的 [LangSmith](https://docs.smith.langchain.com/) API 密钥：

```typescript
// process.env.LANGSMITH_TRACING="true"
// process.env.LANGSMITH_API_KEY="your-api-key"
```

## 实例化

In [1]:
import { RedisVectorStore } from "@langchain/redis";
import { OpenAIEmbeddings } from "@langchain/openai";

import { createClient } from "redis";

const embeddings = new OpenAIEmbeddings({
  model: "text-embedding-3-small",
});

const client = createClient({
  url: process.env.REDIS_URL ?? "redis://localhost:6379",
});
await client.connect();

const vectorStore = new RedisVectorStore(embeddings, {
  redisClient: client,
  indexName: "langchainjs-testing",
});

## 管理向量存储

### 向向量存储中添加条目

In [2]:
import type { Document } from "@langchain/core/documents";

const document1: Document = {
  pageContent: "The powerhouse of the cell is the mitochondria",
  metadata: { type: "example" }
};

const document2: Document = {
  pageContent: "Buildings are made out of brick",
  metadata: { type: "example" }
};

const document3: Document = {
  pageContent: "Mitochondria are made out of lipids",
  metadata: { type: "example" }
};

const document4: Document = {
  pageContent: "The 2024 Olympics are in Paris",
  metadata: { type: "example" }
}

const documents = [document1, document2, document3, document4];

await vectorStore.addDocuments(documents);

顶级文档ID和删除当前不受支持。

## 查询向量存储

一旦创建了向量存储并添加了相关文档，在运行链或代理时很可能需要对其进行查询。

### 直接查询

执行一个简单的相似性搜索可以按照以下方式：

In [7]:
const similaritySearchResults = await vectorStore.similaritySearch("biology", 2);

for (const doc of similaritySearchResults) {
  console.log(`* ${doc.pageContent} [${JSON.stringify(doc.metadata, null)}]`);
}

当前过滤将查找包含提供的字符串的任何元数据键。

如果您要执行相似性搜索并接收相应的分数，您可以运行：

In [8]:
const similaritySearchWithScoreResults = await vectorStore.similaritySearchWithScore("biology", 2)

for (const [doc, score] of similaritySearchWithScoreResults) {
  console.log(`* [SIM=${score.toFixed(3)}] ${doc.pageContent} [${JSON.stringify(doc.metadata)}]`);
}

* [SIM=0.835] The powerhouse of the cell is the mitochondria [{"type":"example"}]
* [SIM=0.852] Mitochondria are made out of lipids [{"type":"example"}]


### 通过转换为检索器进行查询

你还可以将向量存储转换为[检索器](/docs/concepts/retrievers)，以便在你的链中更方便地使用。

In [9]:
const retriever = vectorStore.asRetriever({
  k: 2,
});
await retriever.invoke("biology");

[
  Document {
    pageContent: 'The powerhouse of the cell is the mitochondria',
    metadata: { type: 'example' },
    id: undefined
  },
  Document {
    pageContent: 'Mitochondria are made out of lipids',
    metadata: { type: 'example' },
    id: undefined
  }
]


### 检索增强生成的用法

有关如何将此向量存储用于检索增强生成（RAG）的指南，请参阅以下部分：

- [教程：使用外部知识](/docs/tutorials/#working-with-external-knowledge)。
- [操作指南：使用RAG进行问答](/docs/how_to/#qa-with-rag)
- [检索概念文档](/docs/concepts/retrieval)

## 删除索引

您可以使用以下命令删除整个索引：

In [10]:
await vectorStore.delete({ deleteAll: true });

## 关闭连接

完成操作后，请确保关闭客户端连接，以避免资源过度消耗：

In [11]:
await client.disconnect();

## API 参考文档

如需详细了解所有 `RedisVectorSearch` 功能和配置，请前往 [API 参考文档](https://api.js.langchain.com/classes/langchain_redis.RedisVectorStore.html)。